In [15]:
from tool import *
import plotly.graph_objects as go
import plotly.express as px

# 업체별 선적 실적

# 모니터 2023년 선적 현황

In [2]:
with open('D:/Data/ODM_SR.db', 'rb') as f:
    sr = pickle.load(f)
with open('D:/Data/Shipment Plan_Adjusted_new.DB', 'rb') as f:
    spdb = pickle.load(f)
with open('D:/Data/contact_window.db', 'rb') as f:
    cont = pickle.load(f)
with open('D:/Data/sitemap.db', 'rb') as f:
    site = pickle.load(f)
with open('D:/Data/contact_window.db', 'rb') as f:
    cont = pickle.load(f)

In [3]:
sr = sr.reset_index()
this_week = get_weekname(datetime.datetime.today())
# sr = sr[sr['From Site'].isin(['OS_TCL_CN_P', 'OS_TPV_CN_P', 'OS_AUO_CN_P'])] # 업체 필터링
sr['Week Name'] = sr['Ship Date'].map(get_weekname)
sr[['Year', 'Month']] = sr['Week Name'].apply(lambda x:f'{datetime.date.fromisoformat(x[:10]).isocalendar().year}-{get_month_from_date(datetime.date.fromisoformat(x[:10])):02d}').str.split('-', expand=True)
# sr = sr[sr['Year'] == '2023'] # 2023년 실적만 필터링
sr['Quarter'] = sr['Month'].apply(lambda x:pd.to_datetime(x, format='%m')).dt.to_period('Q').astype(str).apply(lambda x:x[-2:])
sr = sr[sr['Week Name'] <= get_weekname_from(this_week, -1)]
sr['To Site'] = sr['To Site'].replace(ship_to_map)
sr = pd.merge(sr, site[['Region', 'Subsidiary', 'To Site', 'Country']], how='left')

# IT ODM 모델명에서 시리즈명 추출
c1 = sr['From Site'].str.contains('AUO|TPV|TCL|BOEVT') # 모니터 ODM 일 경우
c2 = sr['Mapping Model.Suffix'].str.match('\d{2}[T|Z|U|A][G|D|N|W|C]?\d{2}[A-Z]{1,2}') # PC/태블릿 모델일 경우
c3 = sr['Mapping Model.Suffix'].str.contains('CL600') # 페가트론 CL600 모델일 경우

# sr.loc[c1, 'Series'] = sr[c1]['Mapping Model.Suffix'].apply(lambda x:x[:2]) # 모니터 모델의 시리즈명을 inch 로 추출
sr.loc[c1, 'Series'] = sr[c1]['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).apply(lambda x:x.split('.')[0]) # Mapping 모델명의 앞부분을 시리즈명으로 추출
sr.loc[c2, 'Series'] = sr[c2]['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).str.replace('[D|G|N|W|C]', '').apply(lambda x:x[:6]) # PC모델의 대표모델명 추출
sr.loc[c3, 'Series'] = 'CL600'

sr = sr[['Year', 'Month', 'Quarter', 'From Site', 'Division', 'Region', 'Country', 'Subsidiary', 'To Site', 'Series', 'Mapping Model.Suffix', 'Week Name', 'Ship']]
sr = sr.rename(columns={'Ship':'SP'})

C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_584\2941770503.py:19: FutureWarning: The default value of regex will change from True to False in a future version.
  sr.loc[c2, 'Series'] = sr[c2]['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).str.replace('D|G|N|W|C', '').apply(lambda x:x[:6]) # PC모델의 대표모델명 추출


In [4]:
spdb.groupby(['Planweek', 'Category', 'Created_at']).sum('SP').reset_index().iloc[-16:, ] # 선택할 SP의 Version 목록을 출력

,Planweek,Category,Created_at,SP
130,2024-02-05(W06),Latest26w,202402051504,1017718.0
131,2024-02-05(W06),Latest26w,202402060857,1002329.0
132,2024-02-12(W07),Final26w,202402131354,1134545.0
133,2024-02-12(W07),Final52w,202402140851,1400784.0
134,2024-02-12(W07),Latest26w,202402121709,1129621.0
135,2024-02-12(W07),Latest26w,202402130921,1130892.0
136,2024-02-12(W07),Latest26w,202402150843,1134445.0
137,2024-02-19(W08),Final26w,202402201401,1190717.0
138,2024-02-19(W08),Final52w,202402210937,1432106.0
139,2024-02-19(W08),Latest26w,202402191522,1191421.0


In [5]:
i = 143 # 선택할 SP version 설정

In [6]:
sp = spdb.merge(spdb.groupby(['Planweek', 'Category', 'Created_at']).sum('SP').reset_index().iloc[[i], :3])
sp = pd.merge(sp, site[['To Site', 'Subsidiary', 'Country']], how='left')
sp

,Planweek,Created_at,Division,From Site,Region,To Site,Mapping Model.Suffix,Rep PMS,Category,Frozen,Week Name,SP,Subsidiary,Country
0,2024-02-26(W09),202402281001,MNT,OS_AUO_CN_P,SEA,EATH,24GS50F-B.ATMQ,24GS50F-BB.CTMAMV,Final52w,N,2024-04-15(W16),572.0,LGETH,Thailand
1,2024-02-26(W09),202402281001,MNT,OS_AUO_CN_P,SEA,EATH,24GS50F-B.ATMQ,24GS50F-BB.CTMAMV,Final52w,N,2024-05-06(W19),572.0,LGETH,Thailand
2,2024-02-26(W09),202402281001,MNT,OS_AUO_CN_P,SEA,EATH,24GS50F-B.ATMQ,24GS50F-BB.CTMAMV,Final52w,N,2024-06-03(W23),572.0,LGETH,Thailand
3,2024-02-26(W09),202402281001,MNT,OS_AUO_CN_P,SEA,EATH,24GS50F-B.ATMQ,24GS50F-BB.CTMAMV,Final52w,N,2024-07-01(W27),572.0,LGETH,Thailand
4,2024-02-26(W09),202402281001,MNT,OS_AUO_CN_P,SEA,EATH,24GS50F-B.ATMQ,24GS50F-BB.CTMAMV,Final52w,N,2024-07-29(W31),120.0,LGETH,Thailand
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3776,2024-02-26(W09),202402281001,PC,OS_WINGTECH_CN_P,KOREA,EKHQ,10A30Q-L.AQ28K,10A30Q-L.AQ28K,Final52w,Y,2024-01-08(W02),360.0,LGEKR,Korea
3777,2024-02-26(W09),202402281001,PC,OS_WINGTECH_CN_P,KOREA,EKHQ,10A30Q-L.AQ28K,10A30Q-L.AQ28K,Final52w,Y,2024-01-29(W05),228.0,LGEKR,Korea
3778,2024-02-26(W09),202402281001,PC,OS_WINGTECH_CN_P,KOREA,EKHQ,10A30Q-L.AQ28K,10A30Q-L.AQ28K,Final52w,Y,2024-02-19(W08),310.0,LGEKR,Korea
3779,2024-02-26(W09),202402281001,PC,OS_WINGTECH_CN_P,KOREA,EKHQ,10A30Q-L.AQ28K,10A30Q-L.AQ28K,Final52w,Y,2024-03-04(W10),160.0,LGEKR,Korea


In [7]:
# sp = sp[sp['From Site'].isin(['OS_TCL_CN_P', 'OS_TPV_CN_P', 'OS_AUO_CN_P'])] # 업체 필터링

# IT ODM 모델명에서 시리즈명 추출
c1 = sp['From Site'].str.contains('AUO|TPV|TCL|BOEVT') # 모니터 ODM 일 경우
c2 = sp['Mapping Model.Suffix'].str.match('\d{2}[T|Z|U|A][G|D|N|W|C]?\d{2}[A-Z]{1,2}') # PC/태블릿 모델일 경우
c3 = sp['Mapping Model.Suffix'].str.contains('CL600') # 페가트론 CL600 모델일 경우

# sp.loc[c1, 'Series'] = sp[c1]['Mapping Model.Suffix'].apply(lambda x:x[:2]) # 모니터 모델의 시리즈명(inch) 추출
sp.loc[c1, 'Series'] = sp[c1]['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).apply(lambda x:x.split('.')[0]) # Mapping 모델명의 앞부분을 시리즈명으로 추출
sp.loc[c2, 'Series'] = sp[c2]['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).str.replace('[D|G|N|W|C]', '').apply(lambda x:x[:6]) # PC모델의 대표모델명 추출
sp.loc[c3, 'Series'] = 'CL600'

sp[['Year', 'Month']] = sp['Week Name'].apply(lambda x:f'{datetime.date.fromisoformat(x[:10]).isocalendar().year}-{get_month_from_date(datetime.date.fromisoformat(x[:10])):02d}').str.split('-', expand=True)
sp['Quarter'] = sp['Month'].apply(lambda x:pd.to_datetime(x, format='%m')).dt.to_period('Q').astype(str).apply(lambda x:x[-2:])
sp = sp[sp['Week Name'] >= this_week]

C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_584\3252752946.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  sp.loc[c2, 'Series'] = sp[c2]['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).str.replace('[D|G|N|W|C]', '').apply(lambda x:x[:6]) # PC모델의 대표모델명 추출


In [91]:
# BR 모델을 MR 모델에 통합
sr.loc[sr['Series'].str.contains('24BR'), 'Series'] = '24MR' 
sr.loc[sr['Series'].str.contains('32BR'), 'Series'] = '32MR'
# 25GR 모델 삭제
sr = sr[~sr['Series'].str.contains('25GR')].reset_index(drop=True)

In [134]:
# Series 모델명 통일화 작업
sp = sp[~sp['Series'].str.contains('25GR')].reset_index(drop=True)
sp.loc[sp['Series'].str.contains('22MS|22BR'), 'Series'] = '22MR'
sp.loc[sp['Series'].str.contains('27MC|27BR'), 'Series'] = '27MR'
sp.loc[sp['Series'].str.contains('32MC|32BR'), 'Series'] = '32MR'
sp.loc[sp['Series'].str.contains('24BR'), 'Series'] = '24MR'

In [9]:
tt = pd.concat([sr, sp], join='inner').reset_index(drop=True)
tt

,Year,Month,Quarter,From Site,Division,Region,Country,Subsidiary,To Site,Series,Mapping Model.Suffix,Week Name,SP
0,2023,08,Q3,OS_TCL_CN_P,MNT,EU,Hungary,LGEMK,EEMK,22MR410,22MR410-B.AEUQ,2023-08-14(W33),840.0
1,2023,08,Q3,OS_TCL_CN_P,MNT,EU,Spain,LGEES,EEES,22MR410,22MR410-B.AEUQ,2023-08-28(W35),838.0
2,2023,08,Q3,OS_TCL_CN_P,MNT,MEA,Turkiye,LGETK,EMTK,22MR410,22MR410-B.APDQ,2023-08-28(W35),840.0
3,2023,08,Q3,OS_TCL_CN_P,MNT,SEA,Thailand,LGETH,EATH,22MR410,22MR410-B.ATMQ,2023-08-21(W34),2184.0
4,2023,08,Q3,OS_TCL_CN_P,MNT,SEA,Thailand,LGETH,EATH,22MR410,22MR410-B.ATMQ,2023-08-21(W34),2184.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7715,2024,03,Q1,OS_TPV_CN_P,MNT,SEA,Japan,LGEJP,EJJP,32MR50C,32MR50C-B.AJLQ,2024-03-25(W13),96.0
7716,2024,03,Q1,OS_WINGTECH_CN_P,PC,KOREA,Korea,LGEKR,EKHQ,10A30Q,10A30Q-L.AQ14K,2024-03-04(W10),1280.0
7717,2024,03,Q1,OS_WINGTECH_CN_P,PC,KOREA,Korea,LGEKR,EKHQ,10A30Q,10A30Q-L.AQ1TL,2024-03-18(W12),500.0
7718,2024,03,Q1,OS_WINGTECH_CN_P,PC,KOREA,Korea,LGEKR,EKHQ,10A30Q,10A30Q-L.AQ1UL,2024-03-18(W12),2000.0


In [31]:
tt[tt['From Site'].str.contains('TCL') & tt['Year'].str.contains('2024') & tt['Quarter'].isin(['Q1', 'Q2'])]['SP'].sum()

465585.0

In [25]:
px.bar(tt[tt['From Site'].str.contains('TCL')].groupby(['Year', 'Quarter']).sum('SP').reset_index(), x='SP', y='Year', color='Quarter', category_orders={'Quarter':['Q1', 'Q2', 'Q3', 'Q4']}, text='Quarter')

In [24]:
px.bar(tt[tt['From Site'].str.contains('TCL')].groupby(['Year', 'Series']).sum('SP').reset_index(), x='SP', y='Year', color='Series', category_orders={'Year':['2023', '2024']}, text='Series')

In [23]:
px.bar(tt[tt['From Site'].str.contains('TCL')].groupby(['Year', 'Region']).sum('SP').reset_index(), x='SP', y='Year', color='Region', category_orders={'Year':['2023', '2024']}, text='Region')

In [85]:
tt[tt['Country'] == 'Canada'].pivot_table('SP', index=['From Site', 'Series', 'Mapping Model.Suffix'], columns=['Year', 'Quarter',], aggfunc=sum).loc['OS_TCL_CN_P'].dropna(how='all', axis=1).fillna('-')

Year                           2023    2024       
Quarter                          Q4      Q1     Q2
Series Mapping Model.Suffix                       
22     22MR410-B.AUSQ        1680.0       -  840.0
       22MR41A-B.ACCQ         840.0   840.0      -
       22MR41S-B.ACCQ         840.0  1680.0  840.0
       22MS31W-B.ACCQ             -       -  840.0
27     27MC41D-BS.ACCQ            -  5500.0  750.0
       27MR400-B.AUSQ        2816.0  2304.0  512.0
       27MR41A-B.ACCQ        1280.0  1536.0      -
       27MR41S-B.ACCQ        1280.0  1024.0  512.0

In [29]:
tt['Region'].unique()

array(['EU', 'MEA', 'SEA', 'NA', 'INDIA', 'SCA', 'KOREA'], dtype=object)

In [86]:
region_korean_map = {'NA':'북미', 'EU':'유럽', 'MEA':'중아', 'SEA':'아시아', 'INDIA':'인도', 'SCA':'중남미', 'KOREA':'한국'}

In [87]:
tt['Region'] = tt['Region'].map(region_korean_map)

In [19]:
tt.to_clipboard()